# load mats into netcdfs from plume tracker

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import xarray as xr

from utils import create_gif, load_pts_mat
from parcels_utils import xr_dataset_to_fieldset
from plot_utils import get_carree_axis, plot_particles

In [ ]:
TIJUANA_RIVER_DOMAIN = dict(
    S=32.528,
    N=32.71,
    W=-117.29,
    E=-117.11,
)
CLOSE_TIJUANA_DOMAIN = dict(
    S=32.53,
    N=32.5825,
    W=-117.162,
    E=-117.105,
)
THING_DOMAIN = {
    "S": 32.41,
    "N": 32.7,
    "W": -117.25,
    "E": -117
}

coastline = load_pts_mat("mat/coastline.mat", "latz0", "lonz0")
# just filter out the problematic upper coastline where it goes up and down
coastline = coastline[:, :288]

In [ ]:
plt.scatter(coastline[1], coastline[0], s=4)

In [ ]:
def coord_inland(coastline, lat, lon):
    lower_ind = np.where(coastline[0] <= lat)[0]
    if lower_ind.size > 0:
        lower_ind = lower_ind[-1]
    else:
        lower_ind = 0
    upper_ind = np.where(coastline[0] >= lat)[0]
    if upper_ind.size > 0:
        upper_ind = upper_ind[0]
    else:
        upper_ind = -1
    left = min(coastline[1][lower_ind], coastline[1][upper_ind])
    # can get away with this since the coastline points are dense enough
    return lon > left


def parse_mat_time(time):
    date = np.datetime64("0") + np.timedelta64(int(time) - 1, "D")
    hour = int(round((time - int(time)) * 24))
    return date + np.timedelta64(hour, "h")


def dataset_from_mat(grid_mat_path, current_mat_path, remove_land_currents):
    grid_mat = scipy.io.loadmat(grid_mat_path)
    current_mat = scipy.io.loadmat(current_mat_path)
    coords = grid_mat["totalGrid"]
    u = current_mat["U"]
    v = current_mat["V"]
    time = np.array([parse_mat_time(current_mat["t"][0, 0])])
    lats = np.sort(np.unique(coords.T[1]))
    lons = np.sort(np.unique(coords.T[0]))
    u_grid = np.zeros((1, len(lats), len(lons)))
    v_grid = np.zeros((1, len(lats), len(lons)))
    for i in range(len(lons)):
        for j in range(len(lats)):
            if remove_land_currents and not np.isnan(u[0][i * len(lats) + j]) and coord_inland(coastline, lats[j], lons[i]):
                u_grid[0, j, i] = np.nan
                v_grid[0, j, i] = np.nan
            else:
                # data from .mat is measured in cm/s
                u_grid[0, j, i] = u[0][i * len(lats) + j] / 100
                v_grid[0, j, i] = v[0][i * len(lats) + j] / 100
    ds = xr.Dataset(
        {
            "u": (["time", "lat", "lon"], u_grid),
            "v": (["time", "lat", "lon"], v_grid),
        },
        coords={
            "time": time,
            "lat": lats,
            "lon": lons
        }
    )
    return ds

In [ ]:
file_prefix = "Tot_SDLJ_20200810"
start = 0
incr = 100

In [ ]:
ds_total = None
for i in range(start, 2400, incr):
    name = f"mat/{file_prefix}_{str(i).zfill(4)}.mat"
    try:
        ds = dataset_from_mat("mat/codartotalGrid.mat", name, True)
        if ds_total is None:
            ds_total = ds
        else:
            ds_total = xr.concat([ds_total, ds], dim="time")
        data = scipy.io.loadmat(name)
    except FileNotFoundError:
        print(f"file {name} not found")

In [ ]:
ds_total.to_netcdf(f"current_netcdfs/{file_prefix}.nc")

In [ ]:
fs = xr_dataset_to_fieldset(ds_total)
fs.U.show()